# Burrows-Wheeler Transform

In [1]:
def rotations(t):
    # Return list of rotations of input string t
    tt = t * 2
    return [ tt[i:i+len(t)] for i in range(len(t)) ]

In [2]:
rotations('abaaba$')

['abaaba$', 'baaba$a', 'aaba$ab', 'aba$aba', 'ba$abaa', 'a$abaab', '$abaaba']

In [3]:
def bwm(t):
    # Return lexicographically sorted list of t’s rotations
    return sorted(rotations(t))

In [4]:
print('\n'.join(bwm('abaaba$')))

$abaaba
a$abaab
aaba$ab
aba$aba
abaaba$
ba$abaa
baaba$a


In [5]:
def bwtViaBwm(t):
    # Given T, returns BWT(T) by way of the BWM
    return ''.join(map(lambda x: x[-1], bwm(t)))

In [6]:
bwtViaBwm('abaaba$') # we can see the result equals the last column of the matrix above

'abba$aa'

In [7]:
bwtViaBwm('Tomorrow_and_tomorrow_and_tomorrow$')

'w$wwdd__nnoooaattTmmmrrrrrrooo__ooo'

In [8]:
bwtViaBwm('It_was_the_best_of_times_it_was_the_worst_of_times$')

's$esttssfftteww_hhmmbootttt_ii__woeeaaressIi_______'

In [9]:
bwtViaBwm('in_the_jingle_jangle_morning_Ill_come_following_you$')

'u_gleeeengj_mlhl_nnnnt$nwj__lggIolo_iiiiarfcmylo_oo_'

## Relation between BWT and Suffix Array

In [10]:
def suffixArray(s):
    '''
    Given T return suffix array SA(T). We use Python's sorted
    function here for simplicity, but we can do better.
    '''
    satups = sorted([(s[i:], i) for i in range(len(s))])
    # Extract and return just the offsets
    return map(lambda x: x[1], satups)

def bwtViaSa(t):
    # Given T, returns BWT(T) by way of the suffix array
    bw = []
    for si in suffixArray(t):
        if si == 0:
            bw.append('$')
        else:
            bw.append(t[si-1])
    return ''.join(bw) # return string-ized version of list bw

In [11]:
print(bwtViaBwm('abaaba$'))
print(bwtViaSa('abaaba$'))

abba$aa
abba$aa


In [12]:
print(bwtViaBwm('Tomorrow_and_tomorrow_and_tomorrow$'))
print(bwtViaSa('Tomorrow_and_tomorrow_and_tomorrow$'))

w$wwdd__nnoooaattTmmmrrrrrrooo__ooo
w$wwdd__nnoooaattTmmmrrrrrrooo__ooo


In [13]:
print(bwtViaBwm('It_was_the_best_of_times_it_was_the_worst_of_times$'))
print(bwtViaSa('It_was_the_best_of_times_it_was_the_worst_of_times$'))

s$esttssfftteww_hhmmbootttt_ii__woeeaaressIi_______
s$esttssfftteww_hhmmbootttt_ii__woeeaaressIi_______


In [14]:
print(bwtViaBwm('in_the_jingle_jangle_morning_Ill_come_following_you$'))
print(bwtViaSa('in_the_jingle_jangle_morning_Ill_come_following_you$'))

u_gleeeengj_mlhl_nnnnt$nwj__lggIolo_iiiiarfcmylo_oo_
u_gleeeengj_mlhl_nnnnt$nwj__lggIolo_iiiiarfcmylo_oo_


## BWT Reversing

In [15]:
t = 'abaaba$'
b = bwtViaBwm(t)
b

'abba$aa'

In [17]:
def rankBwt(bw):
    ''' 
    Given BWT string bw, return parallel list of B-ranks.  Also
    returns tots: map from character to # times it appears. 
    '''
    tots = dict()
    ranks = []
    for c in bw:
        if c not in tots:
            tots[c] = 0
        ranks.append(tots[c])
        tots[c] += 1
    return ranks, tots

In [23]:
ranks, tots = rankBwt(b)
# print characters of BWT(T) in order, along with rank
for i in zip(b, ranks): 
    print(i)
for i in tots:
    print(i, tots[i])

('a', 0)
('b', 0)
('b', 1)
('a', 1)
('$', 0)
('a', 2)
('a', 3)
a 4
b 2
$ 1


In [24]:
def firstCol(tots):
    ''' 
    Return map from character to the range of rows prefixed by
    the character. 
    '''
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [25]:
firstCol(tots)

{'$': (0, 1), 'a': (1, 5), 'b': (5, 7)}

In [26]:
# confirm that the representation of the first column above is sensible
print('\n'.join(bwm(t)))

$abaaba
a$abaab
aaba$ab
aba$aba
abaaba$
ba$abaa
baaba$a


In [27]:
def reverseBwt(bw):
    ''' Make T from BWT(T) '''
    ranks, tots = rankBwt(bw)
    first = firstCol(tots)
    rowi = 0 # start in first row
    t = '$' # start with rightmost character
    while bw[rowi] != '$':
        c = bw[rowi]
        t = c + t # prepend to answer
        # jump to row that starts with c of same rank
        rowi = first[c][0] + ranks[rowi]
    return t

In [28]:
reverseBwt(b)

'abaaba$'

In [29]:
reverseBwt(bwtViaBwm('It_was_the_best_of_times_it_was_the_worst_of_times$'))

'It_was_the_best_of_times_it_was_the_worst_of_times$'